In [1]:
from xarray import DataArray

from weldx import Q_, GenericSeries, LinearHorizontalTraceSegment, Trace
from weldx.core import SpatialSeries
from weldx.geometry import DynamicTraceSegment

In [2]:
lhts = LinearHorizontalTraceSegment("2mm")

In [3]:
tr_1 = Trace(lhts)

In [4]:
tr_1.plot()

DimensionalityError: Cannot convert from 'dimensionless' (dimensionless) to 'millimeter' ([length])

In [ ]:
class SDTraceSegment:
    def __init__(self, sd):
        self._sd = sd

In [ ]:
expr = "a*s+b"
# expr = "x+y+z"
params = dict(
    a=DataArray(Q_([1, 0, 0], "mm"), dims=["c"], coords=dict(c=["x", "y", "z"])),
    b=DataArray(Q_([0, 1, 0], "mm"), dims=["c"], coords=dict(c=["x", "y", "z"])),
)
sps = SpatialSeries(expr, parameters=params)

In [ ]:
sdts = SDTraceSegment(sps)

In [ ]:
sps.evaluate(
    s=DataArray(
        Q_([1], ""), dims=["s"], coords=dict(s=DataArray(Q_([1], ""), dims=["s"]))
    )
)

In [ ]:
t = DataArray([1, 3, 5], dims=["a"])

In [ ]:
t.assign_coords(dict(a=[1, 3, 4]))

In [ ]:
DataArray(
    Q_([1], ""),
    dims=["s"],
    coords=dict(s=DataArray(Q_([1], ""), dims=["s"]).pint.dequantify()),
)

In [ ]:
a = DataArray(Q_([1, 2, 3], "mm"), dims=["c"]).pint.dequantify()

In [ ]:
b = DataArray(Q_([2, 3, 5], ""), dims=["c"], coords=dict(c=a))

In [ ]:
print(b.coords["c"])